# Clover

In [ ]:
#| default_exp 62-clover-ep-for-wikiseealso-with-clr-for-dr

In [ ]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from xcai.basics import *
from xcai.models.PPP0XX import DBT013, DBT014

## Data

In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['WANDB_PROJECT']='xc-nlg_22-oak-training-pipeline-with-multitriplet-loss-and-clustering'

In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC_NLG/data'

In [ ]:
block = XCBlock.from_cfg(data_dir, 'data', valid_pct=0.001, tfm='xcnlg', 
                         tokenizer='distilbert-base-uncased', smp_features=[('lbl2data',1,2)])

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
#| export
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data_distilbert-base-uncased_xcnlg_ngame.pkl'

In [ ]:
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
#| export
with open(pkl_file, 'rb') as file: block = pickle.load(file)

## Training

In [ ]:
#| export
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/62-clover-ep-for-wikiseealso-with-clr-for-dr',
    logging_first_step=True,
    per_device_train_batch_size=700,
    per_device_eval_batch_size=700,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.1,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=10,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
)

In [ ]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT014.from_pretrained('distilbert-base-uncased', ig_tok=0, bsz=bsz, tn_targ=1000, margin=0.3, 
                               tau=0.1, n_negatives=10, apply_softmax=True, lw=1, tie_word_embeddings=False)
model.init_dr_head()
model.use_generation = False

Some weights of DBT014 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight', 'encoder.vocab_projector.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trie = XCTrie.from_block(block)

  0%|          | 0/312330 [00:00<?, ?it/s]

In [ ]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=test_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
learn.train()

[2024-05-31 20:15:36,508] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


node-0:2549730:2549730 [0] NCCL INFO Bootstrap : Using eth0:10.13.60.215<0>
node-0:2549730:2549730 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:2549730:2549730 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:2549730:2549730 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:2549730:2551033 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:2549730:2551033 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:2549730:2551033 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

nod

node-0:2549730:2551034 [1] NCCL INFO Trees [0] -1/-1/-1->1->0 [1] -1/-1/-1->1->0 [2] -1/-1/-1->1->0 [3] 0/-1/-1->1->-1 [4] 0/-1/-1->1->-1 [5] 0/-1/-1->1->-1 [6] -1/-1/-1->1->0 [7] -1/-1/-1->1->0 [8] -1/-1/-1->1->0 [9] 0/-1/-1->1->-1 [10] 0/-1/-1->1->-1 [11] 0/-1/-1->1->-1 [12] -1/-1/-1->1->0 [13] -1/-1/-1->1->0 [14] -1/-1/-1->1->0 [15] 0/-1/-1->1->-1 [16] 0/-1/-1->1->-1 [17] 0/-1/-1->1->-1 [18] -1/-1/-1->1->0 [19] -1/-1/-1->1->0 [20] -1/-1/-1->1->0 [21] 0/-1/-1->1->-1 [22] 0/-1/-1->1->-1 [23] 0/-1/-1->1->-1 comm 0xf0a2ee0 nRanks 02 busId 600000
node-0:2549730:2551033 [0] NCCL INFO Ring 6 : 1 -> 0 -> 1 comm 0x105a4b30 nRanks 02 busId 500000
node-0:2549730:2551033 [0] NCCL INFO Ring 7 : 1 -> 0 -> 1 comm 0x105a4b30 nRanks 02 busId 500000
node-0:2549730:2551033 [0] NCCL INFO Ring 8 : 1 -> 0 -> 1 comm 0x105a4b30 nRanks 02 busId 500000
node-0:2549730:2551034 [1] NCCL INFO P2P Chunksize set to 524288
node-0:2549730:2551033 [0] NCCL INFO Ring 9 : 1 -> 0 -> 1 comm 0x105a4b30 nRanks 02 busId 500

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)
/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/utils/import_utils.py:533: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/.singularity/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2549730/1093534927.py", line 1, in <cell line: 1>
    learn.train()
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/trainer.py", line 1885, in train
    return inner_training_loop(
  File "/home/aiscuser/scratch/Projects/xcai/xcai/learner.py", line 975, in _inner_training_loop
    for step, inputs in enumerate(epoch_iterator):
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 679, in _next_data
    data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", li

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/.singularity/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2549730/1093534927.py", line 1, in <cell line: 1>
    learn.train()
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/trainer.py", line 1885, in train
    return inner_training_loop(
  File "/home/aiscuser/scratch/Projects/xcai/xcai/learner.py", line 975, in _inner_training_loop
    for step, inputs in enumerate(epoch_iterator):
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 679, in _next_data
    data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)
  File "/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", li

TypeError: object of type 'NoneType' has no len()

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()

## Prediction

In [ ]:
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/44-encoder-parallel-clover-for-wikiseealso-1-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    output_concatenation_weight=1.0,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
)

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
output_dir = f"/home/aiscuser/scratch/Projects/xc_nlg/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT013.from_pretrained(mname, ig_tok=0, bsz=bsz, tn_targ=1000, margin=0.3, tau=0.1, n_negatives=5, 
                               apply_softmax=True, lw=0.01, tie_word_embeddings=False)

In [ ]:
trie = XCTrie.from_block(block)

  0%|          | 0/312330 [00:00<?, ?it/s]

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

In [ ]:
model.use_generation = False

In [ ]:
o = learn.predict(block.test.dset)

[2024-06-05 17:04:23,695] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


  0%|          | 0/196 [00:00<?, ?it/s]

node-0:2327013:2327013 [0] NCCL INFO Bootstrap : Using eth0:10.13.51.163<0>
node-0:2327013:2327013 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:2327013:2327013 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:2327013:2327013 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:2327013:2333616 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:2327013:2333616 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:2327013:2333616 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

nod

node-0:2327013:2333617 [1] NCCL INFO Trees [0] -1/-1/-1->1->0 [1] -1/-1/-1->1->0 [2] -1/-1/-1->1->0 [3] 0/-1/-1->1->-1 [4] 0/-1/-1->1->-1 [5] 0/-1/-1->1->-1 [6] -1/-1/-1->1->0 [7] -1/-1/-1->1->0 [8] -1/-1/-1->1->0 [9] 0/-1/-1->1->-1 [10] 0/-1/-1->1->-1 [11] 0/-1/-1->1->-1 [12] -1/-1/-1->1->0 [13] -1/-1/-1->1->0 [14] -1/-1/-1->1->0 [15] 0/-1/-1->1->-1 [16] 0/-1/-1->1->-1 [17] 0/-1/-1->1->-1 [18] -1/-1/-1->1->0 [19] -1/-1/-1->1->0 [20] -1/-1/-1->1->0 [21] 0/-1/-1->1->-1 [22] 0/-1/-1->1->-1 [23] 0/-1/-1->1->-1 comm 0xe687d90 nRanks 02 busId e00000
node-0:2327013:2333617 [1] NCCL INFO P2P Chunksize set to 524288
node-0:2327013:2333616 [0] NCCL INFO Ring 12 : 1 -> 0 -> 1 comm 0xe52d380 nRanks 02 busId d00000
node-0:2327013:2333616 [0] NCCL INFO Ring 13 : 1 -> 0 -> 1 comm 0xe52d380 nRanks 02 busId d00000
node-0:2327013:2333616 [0] NCCL INFO Ring 14 : 1 -> 0 -> 1 comm 0xe52d380 nRanks 02 busId d00000
node-0:2327013:2333616 [0] NCCL INFO Ring 15 : 1 -> 0 -> 1 comm 0xe52d380 nRanks 02 busId d00

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
o.metrics

{'test_loss': 0.10287630558013916,
 'test_P@1': 0.3175675295045489,
 'test_P@10': 0.10198405768532041,
 'test_P@3': 0.21327775117593467,
 'test_P@5': 0.1616122581192004,
 'test_N@1': 0.31756752729415894,
 'test_N@10': 0.34496310353279114,
 'test_N@3': 0.3163929283618927,
 'test_N@5': 0.32681193947792053,
 'test_PSP@1': 0.26218859269362227,
 'test_PSP@10': 0.34617904712229003,
 'test_PSP@3': 0.28255831545594623,
 'test_PSP@5': 0.3043939514275761,
 'test_PSN@1': 0.262188583612442,
 'test_PSN@10': 0.3228844404220581,
 'test_PSN@3': 0.2852972447872162,
 'test_PSN@5': 0.3019985556602478,
 'test_R@200': 0.5509873744873766,
 'test_R@10': 0.39147872017170965,
 'test_R@100': 0.5229981224396407,
 'test_runtime': 120.473,
 'test_samples_per_second': 1473.484,
 'test_steps_per_second': 0.921}

In [ ]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,31.7568,21.3278,16.1612,10.1984,31.7568,31.6393,32.6812,34.4963,26.2189,28.2558,30.4394,34.6179,26.2189,28.5297,30.1999,32.2884,39.1479,52.2998,55.0987,0.1029,120.473,1473.484,0.921
